In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# # Set up Chrome options (you can add more options as needed)
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode

# # Initialize WebDriver (this automatically handles the WebDriver installation)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# # URL of the blog
# url = "https://vitalik.eth.limo/general/2024/09/28/alignment.html"

# # Open the webpage
# driver.get(url)

# # Wait for the page to load and for the blog content to be visible
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#doc.container-fluid.markdown-body")))

# # Extract the blog title
# title = driver.find_element(By.CSS_SELECTOR, "h1").text  # Assuming the title is in an <h1> tag

# # Extract the blog content
# blog_content = driver.find_element(By.CSS_SELECTOR, "div#doc.container-fluid.markdown-body").text

# # Remove redundant title, date, and other unnecessary parts
# cleaned_content = blog_content.replace(title, "").replace("Dark Mode Toggle", "").replace("See all posts", "").strip()

# # Highlight the title and combine it with the cleaned content
# final_content = f"??? - Blog Title: {title}\n\n{cleaned_content}"

# # Save the cleaned text to a .txt file
# with open("blog_content.txt", "w", encoding="utf-8") as file:
#     file.write(final_content)

# # Close the driver
# driver.quit()

# print("Blog content saved to 'blog_content.txt'")


In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# File paths
input_file = "links.txt"  # Replace with your .txt file containing links
output_file = "aggregated_blog_content.txt"

# Set up Chrome options (headless mode)
chrome_options = Options()
chrome_options.add_argument("--headless")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Function to clean up content and remove excess blank lines
def clean_content(content):
    # Replace more than two consecutive newlines with exactly two
    return re.sub(r"\n{3,}", "\n\n", content)

# Function to process a single link and return cleaned content
def process_link(link):
    try:
        # Open the webpage
        driver.get(link.strip())

        # Wait for the content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#doc.container-fluid.markdown-body"))
        )

        # Extract title
        title = driver.find_element(By.CSS_SELECTOR, "h1").text

        # Extract blog content
        blog_content = driver.find_element(By.CSS_SELECTOR, "div#doc.container-fluid.markdown-body").text

        # Clean and format the content
        cleaned_content = blog_content.replace(title, "").replace("Dark Mode Toggle", "").replace("See all posts", "").strip()
        final_content = f"??? Blog Title: {title}\n\n{clean_content(cleaned_content)}\n\n{'-'*80}\n\n"
        
        return final_content

    except Exception as e:
        print(f"Error processing link {link}: {e}")
        return None

# Open the links file and read all links
with open(input_file, "r", encoding="utf-8") as file:
    links = file.readlines()

# Open the output file in append mode
with open(output_file, "a", encoding="utf-8") as file:
    for link in links:
        print(f"Processing link: {link.strip()}")
        content = process_link(link)
        if content:
            file.write(content)

# Close the WebDriver
driver.quit()

print(f"All content has been aggregated and saved to {output_file}")


Processing link: https://vitalik.eth.limo/general/2024/08/21/plurality.html
Processing link: https://vitalik.eth.limo/general/2024/08/03/museumfuture.html
Processing link: https://vitalik.eth.limo/general/2024/07/23/circlestarks.html
Processing link: https://vitalik.eth.limo/general/2024/07/17/procrypto.html
Processing link: https://vitalik.eth.limo/general/2024/06/30/epochslot.html
Processing link: https://vitalik.eth.limo/general/2024/05/31/blocksize.html
Processing link: https://vitalik.eth.limo/general/2024/05/29/l2culture.html
Processing link: https://vitalik.eth.limo/general/2024/05/23/l2exec.html
Processing link: https://vitalik.eth.limo/general/2024/05/17/decentralization.html
Processing link: https://vitalik.eth.limo/general/2024/05/09/multidim.html
Processing link: https://vitalik.eth.limo/general/2024/04/29/binius.html
Processing link: https://vitalik.eth.limo/general/2024/04/01/dc.html
Processing link: https://vitalik.eth.limo/general/2024/03/29/memecoins.html
Processing li